In [1]:
import numpy as np
import cv2
import heapq
from queue import PriorityQueue

segmented_mask = cv2.imread("unet_mask_855.png", cv2.IMREAD_GRAYSCALE)

segmented_mask = (segmented_mask > 128).astype(np.uint8)

# Define movement directions (4-way or 8-way)
DIRECTIONS = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # 4-way movement

class Graph:
    def __init__(self, mask):
        self.mask = mask
        self.rows, self.cols = mask.shape

    def neighbors(self, node):
        x, y = node
        neighbors = []
        for dx, dy in DIRECTIONS:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.rows and 0 <= ny < self.cols and self.mask[nx, ny] == 1:
                neighbors.append((nx, ny))
        return neighbors

    def cost(self, node1, node2):
        return 1  # Uniform cost for simplicity

# Manhattan heuristic for A*
def manhattan_heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

# Bidirectional A* function
def bidirectional_a_star(graph, start, goal):
    frontier_1 = PriorityQueue()
    frontier_1.put((0, start))
    frontier_2 = PriorityQueue()
    frontier_2.put((0, goal))

    came_from_1, cost_so_far_1 = {start: None}, {start: 0}
    came_from_2, cost_so_far_2 = {goal: None}, {goal: 0}

    while not frontier_1.empty() and not frontier_2.empty():
        _, current_1 = frontier_1.get()
        if current_1 in cost_so_far_2:
            return reconstruct_path(came_from_1, start, current_1) + reconstruct_path(came_from_2, goal, current_1)[1:]

        _, current_2 = frontier_2.get()
        if current_2 in cost_so_far_1:
            return reconstruct_path(came_from_1, start, current_2) + reconstruct_path(came_from_2, goal, current_2)[1:]

        for next in graph.neighbors(current_1):
            new_cost = cost_so_far_1[current_1] + graph.cost(current_1, next)
            if next not in cost_so_far_1 or new_cost < cost_so_far_1[next]:
                cost_so_far_1[next] = new_cost
                priority = new_cost + manhattan_heuristic(goal, next)
                frontier_1.put((priority, next))
                came_from_1[next] = current_1

        for next in graph.neighbors(current_2):
            new_cost = cost_so_far_2[current_2] + graph.cost(current_2, next)
            if next not in cost_so_far_2 or new_cost < cost_so_far_2[next]:
                cost_so_far_2[next] = new_cost
                priority = new_cost + manhattan_heuristic(start, next)
                frontier_2.put((priority, next))
                came_from_2[next] = current_2

    return "No path found"

def reconstruct_path(came_from, start, current):
    path = []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start)
    return path[::-1]

# Example start and goal points (manually selected water pixels)
start = (10, 10)  # Adjust based on actual water path
goal = (100, 100)

# Run pathfinding
graph = Graph(segmented_mask)
path = bidirectional_a_star(graph, start, goal)

# Draw path on the image
for x, y in path:
    segmented_mask[x, y] = 127  # Mark path in gray

cv2.imwrite("path_result.png", segmented_mask)
print("Path saved as path_result.png")


[ WARN:0@0.005] global loadsave.cpp:268 findDecoder imread_('unet_mask_855.png'): can't open/read file: check file path/integrity


TypeError: '>' not supported between instances of 'NoneType' and 'int'